In [1]:
import numpy as np
import pandas as pd
a=pd.read_csv('/content/tmdb_5000_credits.csv')
b=pd.read_csv('/content/tmdb_5000_movies.csv')
#a.head(1)

In [2]:
#now we will merge the two data sets to make analysis easy
x=a.merge(b,on='title')
x.head(1)


,movie_id,title,cast,crew,budget,genres,homepage,id,keywords,original_language,...,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,...,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,7.2,11800


In [3]:
# now we will only take the columns in account which are necessary for the recommendation system
x=x[['movie_id','title','cast','crew','overview','keywords','genres']]
#x.head(1)

In [4]:
x.isnull().sum()
#data preprocessing

,0
movie_id,0
title,0
cast,0
crew,0
overview,3
keywords,0
genres,0


In [5]:
x=x.dropna()


In [6]:
x.isnull().sum()


,0
movie_id,0
title,0
cast,0
crew,0
overview,0
keywords,0
genres,0


In [7]:
x.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [8]:
import ast
def extract(x):
  l=[]
  for i in ast.literal_eval(x):
    l.append(i['name'])
  return l

In [9]:
x['genres'].apply(extract)

,genres
0,"[Action, Adventure, Fantasy, Science Fiction]"
1,"[Adventure, Fantasy, Action]"
2,"[Action, Adventure, Crime]"
3,"[Action, Crime, Drama, Thriller]"
4,"[Action, Adventure, Science Fiction]"
...,...
4804,"[Action, Crime, Thriller]"
4805,"[Comedy, Romance]"
4806,"[Comedy, Drama, Romance, TV Movie]"
4807,[]


In [10]:
x['genres']=x['genres'].apply(extract)
x['keywords']=x['keywords'].apply(extract)

In [11]:
#x.iloc[0].keywords

In [12]:
x.head(1)

,movie_id,title,cast,crew,overview,keywords,genres
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]"


In [13]:
def extract2(x):
  l=[]
  counter=0
  for i in ast.literal_eval(x):
    if counter!=3:
      l.append(i['name'])
      counter+=1
    else:
      break
  return l

In [14]:
x['cast']=x['cast'].apply(extract2)

In [15]:
x.head(1)


,movie_id,title,cast,crew,overview,keywords,genres
0,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]"


In [16]:
x.iloc[0].crew

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [17]:
def extract3(x):
  l=[]
  for i in ast.literal_eval(x):
    if i['job']=='Director':
      l.append(i['name'])
      break
  return l


In [18]:
x['crew']=x['crew'].apply(extract3)

In [19]:
x.head(1)

,movie_id,title,cast,crew,overview,keywords,genres
0,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]"


In [20]:
x['overview']=x['overview'].apply(lambda x:x.split())


In [21]:
x['cast']=x['cast'].apply(lambda x:[i.replace(" ","") for i in x])
x['crew']=x['crew'].apply(lambda x:[i.replace(" ","") for i in x])
x['keywords']=x['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
x['genres']=x['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [22]:
x.head(1)

,movie_id,title,cast,crew,overview,keywords,genres
0,19995,Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[Action, Adventure, Fantasy, ScienceFiction]"


In [23]:
x['tags']=x['overview']+x['genres']+x['keywords']#+x['cast']+x['crew']

In [24]:
df=x[['movie_id','title','tags']]
df.head(1)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [25]:
df['tags']=df['tags'].apply(lambda x:" ".join(x))
df['tags']=df['tags'].apply(lambda x:x.lower())
df['tags']

<ipython-input-25-bac784cb32a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']=df['tags'].apply(lambda x:" ".join(x))
<ipython-input-25-bac784cb32a1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']=df['tags'].apply(lambda x:x.lower())


,tags
0,"in the 22nd century, a paraplegic marine is di..."
1,"captain barbossa, long believed to be dead, ha..."
2,a cryptic message from bond’s past sends him o...
3,following the death of district attorney harve...
4,"john carter is a war-weary, former military ca..."
...,...
4804,el mariachi just wants to play his guitar and ...
4805,a newlywed couple's honeymoon is upended by th...
4806,"""signed, sealed, delivered"" introduces a dedic..."
4807,when ambitious new york attorney sam is sent t...


In [26]:
df.iloc[0].tags

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

In [27]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

df['tags']=df['tags'].apply(stem)


<ipython-input-27-6a4bdbef8a78>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']=df['tags'].apply(stem)


In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=10000,stop_words='english',token_pattern=r'[a-zA-Z]+')
vectors=cv.fit_transform(df['tags']).toarray()

In [29]:
cv.get_feature_names_out()

array(['aaron', 'aba', 'abandon', ..., 'zoo', 'zooey', 'zookeep'],
      dtype=object)

In [30]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)

In [31]:
def recommend(movie):
  movie_index=df[df['title']==movie].index[0]
  distances=similarity[movie_index]
  movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:16]
  for i in movies_list:
    print(df.iloc[i[0]].title)

In [32]:
recommend('Harry Potter and the Half-Blood Prince')

Harry Potter and the Order of the Phoenix
Harry Potter and the Goblet of Fire
Harry Potter and the Chamber of Secrets
Harry Potter and the Philosopher's Stone
Harry Potter and the Prisoner of Azkaban
Inkheart
The Ant Bully
The Greatest Game Ever Played
The Adventurer: The Curse of the Midas Box
The Indian in the Cupboard
Stardust
The NeverEnding Story
Rotor DR1
Night at the Museum: Secret of the Tomb
Alexander and the Terrible, Horrible, No Good, Very Bad Day


In [33]:
import pickle
pickle.dump(df,open('movies.pkl','wb'))

In [34]:
pickle.dump(similarity,open('similarity.pkl','wb'))
